<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 527 Homework 3

### Problem 0 (3 points)

## Part 1: Line Search 

### Problem 1.1 (Exam Style)

$$\nabla f(x,y) = \begin{bmatrix} ax \\ by \end{bmatrix} $$ 

### Problem 1.2 (Exam Style)

$$h(\alpha) = f(x_0-\alpha ax, y_0 - \alpha b y) = \frac{1}{2} (a(x_0-\alpha ax_0)^2 + b(y_0-\alpha by_0)^2)$$

### Problem 1.3 (Exam Style)

$$h(\alpha) = j\alpha^2 + k\alpha+l$$

$$\text{ArgMin of paraboloid at} \frac{-k}{2j}$$

$$a = a^3x_0^2 + b^3y_0^2$$

$$b = -2a^2x_0^2 + -2b^2y_0^2$$

$$\alpha_1 = \frac{a^3x_0^2 + b^3y_0^2}{a^2x_0^2 + 2b^2y_0^2}$$

### Problem 1.4 (Exam Style)

$$\begin{bmatrix} x_1 \\ y_1 \end{bmatrix} = \begin{bmatrix} x_0 \\ y_0 \end{bmatrix} - \frac{a^2+b^2}{a^3+b^3}(x_0^2+y_0^2)\begin{bmatrix} ax_0 \\ by_0 \end{bmatrix}$$

### Problem 1.5 (Exam Style)

## Part 2: Automatic Differentiation

In [1]:
import autograd.numpy as np
from autograd import grad

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline

In [9]:
def banana(z):
    return np.array(100 * (z[1] - z[0] ** 2) ** 2 + (1 - z[0]) ** 2)

### Problem 2.1

### Problem 2.2 (Exam style except for the function calls)

## Part 3: Gradient Descent

In [13]:
from scipy import optimize
from numpy import linalg as npl


def line_search(f, g, z0, f0, g0, state=None):
    outcome = optimize.line_search(f, g, z0, -g0, g0, f0)
    alpha, f1 = outcome[0], outcome[3]
    evaluations = outcome[1]
    if alpha is None:
        alpha, f1 = 0., f0
    z1 = z0 - g0 * alpha
    return z1, f1, evaluations, state

In [14]:
def gd(f, g, z0, step_function=line_search, state=None, max_evaluations=1000,
       min_step=1.e-8, min_gradient=1.e-6):
    evaluations, h = 0, []
    while True:
        f0, g0 = f(z0), g(z0)
        if not len(h):
            h.append((z0, f0))
        evaluations += 1
        if npl.norm(g0) < min_gradient:
            z1, f1 = z0, f0
            break
        z1, f1, n_eval, state = step_function(f, g, z0, f0, g0, state=state)
        evaluations += n_eval
        h.append((z1, f1))
        if npl.norm(z1 - z0) < min_step or evaluations > max_evaluations:
            break
        z0 = z1
    return z1, f1, evaluations, h

In [15]:
def momentum(f, g, z0, f0, g0, state=None):
    if state is None:
        state = {'alpha': 0.001, 'v0': 0., 'mu': 0.9}
    v1 = state['mu'] * state['v0'] - g0 * state['alpha']
    z1 = z0 + v1
    state['v0'] = v1
    return z1, f(z1), 1, state

### Problem 3.1

In [16]:
z_0 = np.array((-1.2, 1.))
z_star = np.array([1., 1.])

In [17]:
# steps = ((line_search, None), (fixed, {'alpha': 0.001}), (fixed, {'alpha': 0.005}),
#          (momentum, {'alpha': 0.001, 'v0': 0., 'mu': 0.2}),
#          (momentum, {'alpha': 0.001, 'v0': 0., 'mu': 0.5}),
#          (momentum, {'alpha': 0.001, 'v0': 0., 'mu': ?}))

In [18]:
from numpy import array2string


def plot_contours(f, z_ast, rect):
    assert len(z_ast) == 2, 'can only plot in two dimensions'
    n = 101
    xs = np.linspace(rect[0], rect[1], n)
    ys = np.linspace(rect[2], rect[3], n)
    fs = np.array([[f(np.array([x, y])) for x in xs] for y in ys])
    plt.contour(xs, ys, fs, 20, colors='grey', linewidths=1)
    plt.plot(z_ast[0], z_ast[1], 'ro')
    plt.axis('scaled')


def fa(a, p=2):
    return array2string(np.array(a), precision=p, separator=', ')

In [19]:
def plot_history_path(h, f, z_ast, n_evals, state, rect=(-2., 2., -2., 2.)):
    assert len(z_ast) == 2, 'can only plot in two dimensions'
    plot_contours(f, z_ast, rect)
    plt.plot([p[0][0] for p in h], [p[0][1] for p in h])
    plt.plot(h[0][0][0], h[0][0][1], 'bo')
    z_last, f_last = h[-1][0], h[-1][1]
    ft = 'f({}) = {:.2g}; {} steps; {} evals'
    title = ft.format(fa(z_last), f_last, len(h), n_evals)
    if state is not None:
        s = state.copy()
        try:
            del s['v0']
        except KeyError:
            pass
        st = '; '.join(['{}={:.3g}'.format(name, value) for name, value in s.items()])
        title = '\n'.join((title, st))
    plt.title(title)

### Problem 3.2 (Exam Style)